### Set environment

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dateutil.parser import parse
import json
import os.path
from joblib import dump, load
import google.oauth2.credentials
import pandas_gbq
from itertools import zip_longest
import re
import pygsheets
from get_data import bigquery_data
import time

### Start timing & get the current update time

In [ ]:
start = time.perf_counter()
update_time =datetime.fromtimestamp(time.time())
print(f"Update time: {update_time}")

### Load BigQuery credentials

In [ ]:
f = open('user_account.json') #GCP_key
credential = json.load(f)

credentials = google.oauth2.credentials.Credentials(
    credential['token'],
    refresh_token=credential['refresh_token'],
    token_uri=credential['token_uri'],
    client_id=credential['client_id'],
    client_secret=credential['client_secret']
)

pandas_gbq.context.credentials = credentials
bigquery_data.get_bigquery_credential()
f.close()

### Set GCP

In [ ]:
project_id = "database_name"
table_id = 'table_name'

In [ ]:
# remove similarity_value under 70 scores
filter_similarity_df = bigquery_data.get_sql_data("sql/filter_similarity.sql")
filter_similarity_df.to_gbq(
    destination_table=table_id,
    project_id=project_id,
    credentials=credentials,
    if_exists='replace'
)

print("Data uploaded successfully.")

### Get new sf_lead.company

In [6]:
mapping_list_df = bigquery_data.get_sql_data("sql/mapping_list.sql")
mapping_list_df.head(5)

,sf_lead_city,sf_lead_grid,sf_lead_company,sf_account_city,sf_account_grid,sf_account_vendor,distances_meters
0,Taichung City,HRD69Z,甲中盐酥鸡,Taichung City,HAMZW4,九月初生乳脆皮甜甜圈 (大甲南陽店),280.192877
1,Chiayi City,HRLBX3,鑫園飯店,Chiayi City,HA67N2,O_BAR 韓式石鍋拌飯、泡菜鍋,438.374636
2,Chiayi City,HRLBX3,鑫園飯店,Chiayi City,HA86V2,合平鴨肉羹,477.810527
3,Chiayi City,HRLBX3,鑫園飯店,Chiayi City,HWYR5Y,李記捲餅舖,406.778193
4,Chiayi City,HRLBX3,鑫園飯店,Chiayi City,HALXMT,桃埕健康餐盒 (嘉義吳鳳店),241.011414


### Clean Vendor Name

In [ ]:
def clean_text(text):
    cleaned_text = re.sub(r'[^\w\s]', '', str(text))
    cleaned_text = re.sub(' ', '', cleaned_text)
    cleaned_text = cleaned_text.lower()
    return cleaned_text

### sf lead vs sf account highest similarity

In [8]:
from fuzzywuzzy import fuzz

In [ ]:
mapping_list_df['similarity_value'] = mapping_list_df.apply(
    lambda row: fuzz.ratio(
        clean_text(row['sf_lead_company']),
        clean_text(row['sf_account_vendor'])
    ), axis=1
)

mapping_list_df

In [ ]:
mapping_list_df = mapping_list_df.sort_values(by = 'similarity_value' , ascending=False)
mapping_list_df

In [ ]:
# 針對每個 sf_lead_company，選擇相似度最高的記錄
highest_similarity_df = mapping_list_df.loc[
    mapping_list_df.groupby('sf_lead_grid')['similarity_value'].idxmax()
]

# 按照 similarity_value 由大到小排序
highest_similarity_df = highest_similarity_df.sort_values(by='similarity_value', ascending=False).reset_index(drop=True)

# 加入資料更新時間
highest_similarity_df['update_time'] = datetime.fromtimestamp(time.time())

highest_similarity_df

In [ ]:
highest_similarity_df.to_gbq(
    destination_table=table_id,
    project_id=project_id,
    credentials=credentials,
    if_exists='append'
)

### Complete Google API

In [ ]:
end=time.perf_counter()
print("執行時間: %f 秒" % (end-start))

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7092d83a-ef33-4093-ac1b-259d18d53a40' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>